In [1]:
# python 3.8.18

# 🧠 NL2SQL 中文模型訓練與查詢範例
# 本 Notebook 示範如何：
# - 使用 HuggingFace Transformers 微調 T5 模型進行中文轉 SQL 訓練
# - 以中文自然語言問題生成 SQL 語句
# - 查詢 SQLite 資料庫並顯示結果表格

# !pip install transformers sentencepiece datasets accelerate pandas --quiet

In [2]:
# 建立測試資料庫 + 執行 SQL 查詢
import sqlite3
import pandas as pd

# 建立測試 customers 表格
conn = sqlite3.connect("sample_nl2sql.db")
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS customers")
cursor.execute("""CREATE TABLE customers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    city TEXT
)""")

cursor.executemany(
    "INSERT INTO customers (name, city) VALUES (?, ?)",
    [
        ("小明", "台北"),
        ("小美", "台中"),
        ("大雄", "台北"),
        ("靜香", "台南"),
        ("胖虎", "高雄"),
        ("阿信", "新北"),
        ("怡君", "桃園"),
        ("柏翰", "台中"),
        ("冠宇", "新竹"),
        ("雅婷", "台南"),
        ("詠晴", "高雄"),
        ("建志", "台北"),
        ("欣怡", "台中"),
        ("威廷", "嘉義"),
        ("志偉", "花蓮"),
        ("家豪", "台南"),
        ("佩珊", "基隆"),
        ("明秀", "新北"),
        ("柏宏", "桃園"),
        ("志芳", "宜蘭"),
        ("佳怡", "屏東"),
        ("智翔", "台中"),
        ("婉婷", "新竹"),
        ("詠翔", "高雄"),
        ("俊宏", "台東"),
        ("佳蓉", "苗栗"),
        ("宜庭", "台北"),
        ("冠霖", "南投"),
        ("佳玲", "台中"),
        ("惠雯", "嘉義"),
        ("昱廷", "新北"),
        ("沛瑜", "基隆"),
        ("志明", "台南"),
        ("冠瑋", "彰化"),
        ("欣妤", "台中"),
        ("志賢", "桃園"),
        ("柏睿", "高雄"),
        ("怡萱", "台北"),
        ("子芸", "花蓮"),
        ("俊賢", "新竹"),
        ("家榮", "台中"),
        ("宥蓉", "南投"),
        ("品妤", "宜蘭"),
        ("育廷", "屏東"),
        ("湘婷", "新北"),
        ("恩綺", "桃園"),
        ("志華", "高雄"),
        ("怡靜", "台中"),
        ("哲瑋", "新竹"),
        ("嘉容", "苗栗"),
        ("昀潔", "基隆")
     
     ]
)
conn.commit()




In [3]:
# -*- coding: utf-8 -*-
import json, os
from datasets import Dataset
from transformers import T5Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

d:\ProgramData\Anaconda3\envs\python38_cuda_conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 1. 定義你的資料庫 schema
#    對每個 table，列出它的欄位名稱
schemas = {
    "customers": {
        "columns": ["name", "city"]
    }
    # 如果有多張表，就繼續加下去：
    # "orders": {"columns": ["order_id", "customer_id", "amount"]},
}

# 2. 建立訓練資料（同時帶入 schema）
train_data = [
    {"question": "查詢所有顧客的姓名", "query": "SELECT name FROM customers;"},
    {"question": "列出所有住在台北的顧客", "query": "SELECT * FROM customers WHERE city = '台北';"},
    {"question": "查詢所有顧客的姓名和所在城市", "query": "SELECT name, city FROM customers;"},
    {"question": "找出城市為高雄的顧客姓名", "query": "SELECT name FROM customers WHERE city = '高雄';"},
    {"question": "請給我住在台中的顧客名單", "query": "SELECT * FROM customers WHERE city = '台中';"},
    {"question": "顯示所有顧客資訊", "query": "SELECT * FROM customers;"},
    {"question": "列出住在新北的所有顧客姓名", "query": "SELECT name FROM customers WHERE city = '新北';"},
    {"question": "查詢在嘉義的顧客", "query": "SELECT * FROM customers WHERE city = '嘉義';"},
    {"question": "有哪些人住在基隆？", "query": "SELECT name FROM customers WHERE city = '基隆';"},
    {"question": "請列出所有來自桃園的顧客", "query": "SELECT * FROM customers WHERE city = '桃園';"},
    {"question": "找出台南的顧客", "query": "SELECT * FROM customers WHERE city = '台南';"},
    {"question": "列出台東的所有顧客", "query": "SELECT * FROM customers WHERE city = '台東';"},
    {"question": "查詢住在宜蘭的顧客姓名", "query": "SELECT name FROM customers WHERE city = '宜蘭';"},
    {"question": "給我住在南投的所有人名", "query": "SELECT name FROM customers WHERE city = '南投';"},
    {"question": "住在屏東的顧客有哪些？", "query": "SELECT * FROM customers WHERE city = '屏東';"},
    {"question": "苗栗地區有哪些顧客", "query": "SELECT * FROM customers WHERE city = '苗栗';"},
    {"question": "查詢所有顧客的城市", "query": "SELECT city FROM customers;"},
    {"question": "請顯示住在花蓮的所有顧客姓名", "query": "SELECT name FROM customers WHERE city = '花蓮';"},
    {"question": "列出來自新竹的顧客", "query": "SELECT * FROM customers WHERE city = '新竹';"},
    {"question": "哪些顧客來自彰化？", "query": "SELECT name FROM customers WHERE city = '彰化';"},
    {"question": "找出台北的顧客有哪些", "query": "SELECT * FROM customers WHERE city = '台北';"},
    {"question": "顯示來自各城市的顧客資訊", "query": "SELECT name, city FROM customers;"},
    {"question": "查詢所有來自台中的顧客姓名", "query": "SELECT name FROM customers WHERE city = '台中';"},
    {"question": "有哪些顧客是來自高雄", "query": "SELECT * FROM customers WHERE city = '高雄';"},
    {"question": "住在基隆的有誰？", "query": "SELECT name FROM customers WHERE city = '基隆';"},
    {"question": "請列出所有住在新北的顧客資料", "query": "SELECT * FROM customers WHERE city = '新北';"},
    {"question": "列出所有人與他們的城市", "query": "SELECT name, city FROM customers;"},
    {"question": "台南有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '台南';"},
    {"question": "基隆有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '基隆';"},
    {"question": "高雄有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '高雄';"},
    {"question": "新北有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '新北';"},
    {"question": "台中有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '台中';"},
    {"question": "彰化有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '彰化';"},
    {"question": "南投有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '南投';"},
    {"question": "苗栗有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '苗栗';"},
    {"question": "花蓮有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '花蓮';"},
    {"question": "嘉義有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '嘉義';"},
    {"question": "屏東有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '屏東';"},
    {"question": "宜蘭有多少顧客", "query": "SELECT COUNT(*) FROM customers WHERE city = '宜蘭';"},
    {"question": "找出來自台東的所有人", "query": "SELECT name FROM customers WHERE city = '台東';"},
    {"question": "查詢住在宜蘭的所有人", "query": "SELECT * FROM customers WHERE city = '宜蘭';"},
    {"question": "列出來自南投的顧客姓名", "query": "SELECT name FROM customers WHERE city = '南投';"},
    {"question": "顯示桃園的顧客資料", "query": "SELECT * FROM customers WHERE city = '桃園';"},
    {"question": "列出住在屏東的顧客名單", "query": "SELECT name FROM customers WHERE city = '屏東';"},
    {"question": "請給我所有城市為苗栗的顧客", "query": "SELECT * FROM customers WHERE city = '苗栗';"},
    {"question": "花蓮有哪些顧客", "query": "SELECT name FROM customers WHERE city = '花蓮';"},
    {"question": "顯示所有來自新竹的顧客資料", "query": "SELECT * FROM customers WHERE city = '新竹';"},
    {"question": "彰化的顧客有哪些人", "query": "SELECT * FROM customers WHERE city = '彰化';"},
    {"question": "查詢所有顧客名字", "query": "SELECT name FROM customers;"},
    {"question": "列出台北顧客資訊", "query": "SELECT * FROM customers WHERE city = '台北';"},
    {"question": "請顯示台中所有顧客資料", "query": "SELECT * FROM customers WHERE city = '台中';"},
    {"question": "新北有哪些顧客姓名", "query": "SELECT name FROM customers WHERE city = '新北';"},
    {"question": "找出所有住在嘉義的顧客", "query": "SELECT * FROM customers WHERE city = '嘉義';"},
    {"question": "列出所有來自基隆的顧客", "query": "SELECT * FROM customers WHERE city = '基隆';"},
    {"question": "請列出來自花蓮的所有顧客姓名", "query": "SELECT name FROM customers WHERE city = '花蓮';"},
    {"question": "有哪些人來自台東", "query": "SELECT * FROM customers WHERE city = '台東';"},
    {"question": "苗栗的顧客名單", "query": "SELECT name FROM customers WHERE city = '苗栗';"},
    {"question": "住在宜蘭的人有哪些", "query": "SELECT name FROM customers WHERE city = '宜蘭';"},
    {"question": "顯示所有在南投的顧客資料", "query": "SELECT * FROM customers WHERE city = '南投';"},
    {"question": "有哪些顧客是來自屏東的", "query": "SELECT * FROM customers WHERE city = '屏東';"}
]
with open("train.json", "w", encoding="utf-8") as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

In [5]:
# 3. 載入模型與 tokenizer
model_name = "Langboat/mengzi-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name, use_fast=False)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:

# 4. 構造 Dataset 並做 preprocess
with open("train.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)
data = Dataset.from_list(raw_data)

def preprocess(example):
    # 把所有 table(schema) 拼成一個字串
    schema_strs = []
    for table, info in schemas.items():
        cols = ", ".join(info["columns"])
        schema_strs.append(f"{table}({cols})")
    schema_text = " | ".join(schema_strs)

    # 將 question 與 schema 串在一起
    input_text = f"將下列中文問題轉成 SQL：{example['question']} [SEP] 資料表結構：{schema_text}"
    # tokenize
    in_enc = tokenizer(
        input_text,
        truncation=True, padding="max_length", max_length=128
    )
    tgt_enc = tokenizer(
        example["query"],
        truncation=True, padding="max_length", max_length=128
    )
    in_enc["labels"] = tgt_enc["input_ids"]
    return in_enc

tokenized = data.map(preprocess, remove_columns=data.column_names)

Map: 100%|██████████| 60/60 [00:00<00:00, 1819.15 examples/s]


In [7]:

# 5. 訓練設定
args = TrainingArguments(
    output_dir="./sql_model",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    logging_steps=10,
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized
)
trainer.train()

  7%|▋         | 11/150 [00:05<00:35,  3.92it/s]

{'loss': 9.6019, 'grad_norm': 12.290719032287598, 'learning_rate': 4.666666666666667e-05, 'epoch': 0.67}


 14%|█▍        | 21/150 [00:11<00:51,  2.52it/s]

{'loss': 0.7681, 'grad_norm': 2.913479804992676, 'learning_rate': 4.3333333333333334e-05, 'epoch': 1.33}


 20%|██        | 30/150 [00:13<00:27,  4.32it/s]

...

100%|██████████| 150/150 [01:06<00:00,  4.25it/s]

{'loss': 0.004, 'grad_norm': 0.26489928364753723, 'learning_rate': 0.0, 'epoch': 10.0}


100%|██████████| 150/150 [01:13<00:00,  2.05it/s]

{'train_runtime': 73.2607, 'train_samples_per_second': 8.19, 'train_steps_per_second': 2.047, 'train_loss': 0.7237942044933637, 'epoch': 10.0}


TrainOutput(global_step=150, training_loss=0.7237942044933637, metrics={'train_runtime': 73.2607, 'train_samples_per_second': 8.19, 'train_steps_per_second': 2.047, 'total_flos': 102713602867200.0, 'train_loss': 0.7237942044933637, 'epoch': 10.0})

In [8]:

# 6. 儲存 fine-tuned 模型
model.save_pretrained("./sql_model")
tokenizer.save_pretrained("./sql_model")


('./sql_model\\tokenizer_config.json',
 './sql_model\\special_tokens_map.json',
 './sql_model\\spiece.model',
 './sql_model\\added_tokens.json')

In [9]:

# 7. 使用模型進行推論
model = AutoModelForSeq2SeqLM.from_pretrained("./sql_model")
tokenizer = AutoTokenizer.from_pretrained("./sql_model", use_fast=False)

def generate_sql(question: str):
    schema_text = " | ".join(
        f"{t}({', '.join(info['columns'])})" for t, info in schemas.items()
    )
    input_text = f"將下列中文問題轉成 SQL：{question} [SEP] 資料表結構：{schema_text}"
    encoding = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=128)
    outputs = model.generate(**encoding, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# if __name__ == "__main__":
#     q = input("請輸入中文問題：")
#     sql = generate_sql(q)
#     print("→", sql)

In [16]:
# 輸入中文問題
q = input("請輸入中文問題：")
sql = generate_sql(q)
print("Input:", q)
print("SQL:", sql)

# 建立DB連線
conn = sqlite3.connect("sample_nl2sql.db")

# 執行 AI 產生的 SQL 查詢
df = pd.read_sql_query(sql, conn)
conn.close()

print("\n【查詢結果】")
print(df)

Input: 誰住台北
SQL: SELECT * FROM customers WHERE city = '台北';

【查詢結果】
   id name city
0   1   小明   台北
1   3   大雄   台北
2  12   建志   台北
3  27   宜庭   台北
4  38   怡萱   台北
